# Import des outils / jeu de données

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTENC
from keras import layers
from lightgbm import LGBMClassifier
from scipy.stats import boxcox
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import IsolationForest, RandomForestClassifier, VotingClassifier
from sklearn.feature_selection import mutual_info_regression
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    make_scorer,
)
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.naive_bayes import BernoulliNB, ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from tensorflow import keras

In [ ]:
SEED = 0

In [ ]:
np.random.seed(SEED)
sns.set_theme()

In [ ]:
df = pd.read_csv(
    "data/data-cleaned-feature-engineering.csv",
    sep=",",
    index_col="ID",
    parse_dates=True,
)

## Variables globales

In [ ]:
var_numeriques = [
    "Year_Birth",
    "Income",
    "Recency",
    "MntWines",
    "MntFruits",
    "MntMeatProducts",
    "MntFishProducts",
    "MntSweetProducts",
    "MntGoldProds",
    "NumDealsPurchases",
    "NumWebPurchases",
    "NumCatalogPurchases",
    "NumStorePurchases",
    "NumWebVisitsMonth",
]

In [ ]:
var_categoriques = [
    "Education",
    "Marital_Status",
    "Kidhome",
    "Teenhome",
    "AcceptedCmp1",
    "AcceptedCmp2",
    "AcceptedCmp3",
    "AcceptedCmp4",
    "AcceptedCmp5",
    "Response",
]

In [ ]:
LABELS = (0, 1)

## Fonctions et variables utiles

In [ ]:
score_modeles = []

**Tableau.** Informations sur notre classification

|:---------------------------|:----------------------------------|
| **Objectif métier**        | Prédire l'acceptation à une campagne marketing |
| **Problème technique**     | Classification binaire supervisée |
| **Métrique**               | Score F1 sur la classe 1 (clients qui acceptent)<br>À score F1 égal, on choisit la meilleure précision sur la classe 1 |
| **Méthode d'entraînement** | Validation croisée en 5 blocs     |
| **Pré-traitement**         | Variables quantitatives : centrer/réduire<br>Variables qualitatives : OneHot Encoding (Tableau Disjonctif Complet) |
| **Équilibrage des classes**| 1) Aucun<br>2) Sous-échantillonnage aléatoire manuel<br>3) Sur-échantillonnage avec SMOTE |


In [ ]:
def evaluate_models(models, prefix, X_train, X_test, y_train, y_test):
    """Evalue tous les modèles dans `models` et sauvegarde les résultats avec un préfixe `prefix`
    (utile pour distinguer les différentes stratégies de pré-traitement des données)."""
    results = []

    for model, model_name in models:
        name = f"{prefix}/{model_name}"

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        precision = accuracy_score(y_test, y_pred)
        scores = cross_val_score(
            model,
            X_train,
            y_train,
            cv=5,
            scoring=make_scorer(f1_score, labels=[LABELS[1]]),
        )
        scores_mean = scores.mean()
        scores_std = scores.std()

        # clf_report = pd.DataFrame(
        #     classification_report(y_test, y_pred, output_dict=True)
        # ).T
        # cm = confusion_matrix(y_test, y_pred, labels=LABELS, normalize="true")
        # # sns.heatmap(cm, annot=True, cmap="Purples", vmin=0, vmax=1)
        #
        # score_f1_classe1 = clf_report.iloc[1, 2]

        results.append(
            [
                name,
                scores_mean,
                #             scores_std,
            ]
        )
        score_modeles.extend(
            (
                [
                    name,
                    "score_f1_classe1",
                    scores_mean,
                    #               scores_std,
                    precision,
                ],
            )
        )

    return results

# Liste des modèles

**Tableau.** Liste des modèles de notre étude

|:---------------------------|:----------------------------------|
| **Modèles de référence**   | Classificateur Idiot Uniforme (50% de oui et 50% de non)<br>Classificateur Idiot Constant 1 (100% de oui) |
| **Modèles linéaires**  | Régression logistique<br>Analyse Discriminante Linéaire |
| **Arbres de décision**               | Arbre de décision<br>Forêt d'arbres de décision (Random Forest) |
| **Gradient Boosting** | XGBoost<br>LightGBM<br>CatBoost |
| **Machine à vecteurs de support (SVM)** | Classificateur SVM linéaire |
| **k plus proches voisins (k-NN)** | Classificateur k-nn (5 voisins)<br>Classificateur k-nn (15 voisins) |
| **Modèle de vote**| Modèle de "Vote à la majorité" sur 5 modèles :<br>- Régression logistique<br>- Analyse discriminante linéaire<br>- Random Forest<br>- XGBoost<br>- CatBoost |
| **Réseau de neurones**| Réseau de neurones à 5 couches et 1 600 neurones |


In [ ]:
models = [
    [DummyClassifier(strategy="uniform", random_state=SEED), "DummyClassifier_Uniform"],
    [
        DummyClassifier(strategy="constant", constant=1, random_state=SEED),
        "DummyClassifier_Constant1",
    ],
    [LogisticRegression(random_state=SEED), "LogisticRegression"],
    [LinearDiscriminantAnalysis(), "LinearDiscriminantAnalysis"],
    [DecisionTreeClassifier(random_state=SEED), "DecisionTreeClassifier"],
    [RandomForestClassifier(random_state=SEED), "RandomForestClassifier"],
    [xgboost.XGBClassifier(random_state=SEED), "XGBClassifier"],
    [CatBoostClassifier(random_state=SEED, verbose=False), "CatBoostClassifier"],
    [LGBMClassifier(random_state=SEED), "LGBMClassifier"],
    [LinearSVC(random_state=SEED), "LinearSVC"],
    # [BernoulliNB(), "BernoulliNB"],
    # [ComplementNB(), "ComplementNB"],
    [KNeighborsClassifier(), "KNeighborsClassifier5"],
    [KNeighborsClassifier(n_neighbors=15), "KNeighborsClassifier15"],
    [
        VotingClassifier(
            estimators=[
                ("lr", LogisticRegression(random_state=SEED)),
                ("lda", LinearDiscriminantAnalysis()),
                ("dt", RandomForestClassifier(random_state=SEED)),
                ("xgb", xgboost.XGBClassifier(random_state=SEED)),
                ("catboost", CatBoostClassifier(random_state=SEED, verbose=False)),
            ],
            voting="soft",
        ),
        "VotingClassifier",
    ],
]

# Traitement des données

## Pipeline Scaler & OneHotEncoding

In [ ]:
var_cat_non_ohe = [
    "AcceptedCmp1",
    "AcceptedCmp2",
    "AcceptedCmp3",
    "AcceptedCmp4",
    "AcceptedCmp5",
    "Response",
    "HasAcceptedCampaigns",
]

In [ ]:
df["HasAcceptedCampaigns"] = df["HasAcceptedCampaigns"].astype(int)

In [ ]:
var_cat_ohe = [
    "Education",
    "Marital_Status",
    "Kidhome",
    "Teenhome",
    "NbAcceptedCampaigns",
    "NbChildren",
]

In [ ]:
# ohe = OneHotEncoder()

In [ ]:
# ohe.fit_transform(df[var_cat_ohe])

In [ ]:
df[var_categoriques].head()

In [ ]:
preprocessor = ColumnTransformer(
    remainder="passthrough",
    transformers=[
        ("ohe", OneHotEncoder(), var_cat_ohe),
        ("scaler", RobustScaler(), var_numeriques),
    ],
)

In [ ]:
X = df.drop(columns=["Response", "Dt_Customer"])

In [ ]:
preprocessor.fit(X)

In [ ]:
preprocessor.transform(X)

In [ ]:
nouveau_df = pd.DataFrame(preprocessor.transform(X), index=df.index)

In [ ]:
nouveau_df.shape

In [ ]:
# pipeline = Pipeline(steps=[("scaler", RobustScaler())])

# preprocessor = ColumnTransformer(
#     remainder="passthrough",
#     transformers=[
#         ("std", standard_transformer, [3]),
#     ],
# )

In [ ]:
# under_sampling_manuel = Pipeline(steps=[()])

## Par défaut

In [ ]:
y = df[["Response"]].astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    nouveau_df, y, test_size=0.2, random_state=SEED
)

In [ ]:
nouveau_df.shape

In [ ]:
prefix = "défaut"
results = evaluate_models(models, prefix, X_train, X_test, y_train, y_test)

In [ ]:
sorted(results, key=lambda x: x[1], reverse=True)

## Équilibrage des classes

### Under-sampling (manuel)

In [ ]:
# samples0 = df[df["Response"] == 0].sample(350, random_state=SEED)

In [ ]:
# X_eq = pd.concat((samples0, df[df["Response"] == 1]))

In [ ]:
# X_eq["Response"].hist()

In [ ]:
# y_eq = X_eq.pop("Response").astype(int)

In [ ]:
# X_eq = pd.get_dummies(X_eq.drop(columns=["Dt_Customer"]))

In [ ]:
# df["Response"].value_counts()

In [ ]:
NB_A_SUPPRIMER = 1400
drop_indices = np.random.choice(
    nouveau_df[y["Response"] == 0].index, NB_A_SUPPRIMER, replace=False
)
df_subset = nouveau_df.drop(drop_indices)

In [ ]:
y_eq = y.drop(index=drop_indices)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_subset, y_eq, test_size=0.2, random_state=SEED
)

In [ ]:
prefix = "éq_classes"
results = evaluate_models(models, prefix, X_train, X_test, y_train, y_test)

In [ ]:
sorted(results, key=lambda x: x[1], reverse=True)

### Over-sampling (SMOTE)

In [ ]:
cat_cols = list(X.select_dtypes(include=["category", "int", "bool"]).columns)

In [ ]:
cat_cols_index = list(map(lambda c: list(X.columns).index(c), cat_cols))

In [ ]:
list(X.iloc[:, cat_cols_index].columns)

In [ ]:
sm = SMOTENC(
    categorical_features=cat_cols_index,
    random_state=SEED,
)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    nouveau_df, y, test_size=0.2, random_state=SEED
)

In [ ]:
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

In [ ]:
y_train_sm.value_counts(normalize=True)

In [ ]:
prefix = "SMOTE"
results = evaluate_models(models, prefix, X_train_sm, X_test, y_train_sm, y_test)

In [ ]:
sorted(results, key=lambda x: x[1], reverse=True)

# Réseau de neurones

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_subset, y_eq, test_size=0.2, random_state=SEED
)

In [ ]:
X_train = np.asarray(X_train).astype("float32")
y_train = np.asarray(y_train).astype("float32")
X_test = np.asarray(X_test).astype("float32")
y_test = np.asarray(y_test).astype("float32")

In [ ]:
X_train.shape

In [ ]:
np.random.seed(0)
model = keras.Sequential(
    [
        layers.Dense(400, activation="relu", input_shape=[X_train.shape[1]]),
        layers.Dense(400, activation="relu"),
        layers.Dense(400, activation="sigmoid"),
        layers.Dense(400, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["binary_accuracy"],  # "binary_accuracy"
)

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    # validation_split=0.2,
    batch_size=512,
    epochs=1000,
    callbacks=[early_stopping],
    verbose=0,  # hide the output because we have so many epochs
)

In [ ]:
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5
history_df.loc[5:, ["loss", "val_loss"]].plot()
history_df.loc[5:, ["binary_accuracy", "val_binary_accuracy"]].plot()

print(
    ("Best Validation Loss: {:0.4f}" + "\nBest Validation Accuracy: {:0.4f}").format(
        history_df["val_loss"].min(), history_df["val_binary_accuracy"].max()
    )
)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
sns.histplot(y_pred > 0.5, discrete=True)

In [ ]:
y_pred_old = y_pred

In [ ]:
y_pred = y_pred > 0.5

In [ ]:
print(classification_report(y_test, y_pred, labels=LABELS))

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=LABELS)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=LABELS)
disp.plot()

In [ ]:
nom_modele = "Réseau de Neurones"
# ajout_score(model, nom_modele, y_test, y_pred)

# Feature importance

## RandomForest feature importance

In [ ]:
nom_colonnes = preprocessor.get_feature_names_out(X.columns)

In [ ]:
nom_colonnes = list(map(lambda x: x.split("__")[1], nom_colonnes))

In [ ]:
rf = models[6][0]

In [ ]:
fi = rf.feature_importances_

In [ ]:
fi = pd.DataFrame(fi.reshape((1, len(fi))), columns=nom_colonnes)

In [ ]:
fi = fi.sort_values(
    by=0, axis=1, ascending=False
)  # trier les colonnes en fonction de la ligne 0

In [ ]:
plt.figure(figsize=(5, 12))
plt.title("Importance donnée par le modèle RandomForest")
sns.barplot(fi, orient="h", color="gray")

## Permutation importance

In [ ]:
result = permutation_importance(rf, X_test, y_test, n_repeats=10, random_state=SEED)

In [ ]:
# sns.histplot(result.importances_std < 0.015)

In [ ]:
pi_results = result.importances_mean

In [ ]:
pi_results = pd.DataFrame(
    pi_results.reshape((1, len(pi_results))), columns=nom_colonnes
)
pi_results = pi_results.sort_values(by=0, axis=1, ascending=False)

In [ ]:
plt.figure(figsize=(5, 12))
plt.title("Importance de Permutation du modèle RandomForest")
sns.barplot(pi_results, orient="h", color="gray")

# Sauvegarde des données

In [ ]:
score_modeles_df = pd.DataFrame(
    score_modeles, columns=["Modèle", "Métrique", "Valeur", "Précision"]
)

In [ ]:
score_modeles_df.to_csv("data/results/classifications.csv", index=False)